In [4]:
!pip install scikit-image
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr

# Define the Autoencoder Architecture
class ConvAutoencoder(nn.Module):
    def __init__(self, activation_fn):
        super(ConvAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),  # Output: 16x16x16
            activation_fn(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # Output: 8x8x32
            activation_fn(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # Output: 4x4x64
            activation_fn()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # Output: 8x8x32
            activation_fn(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),  # Output: 16x16x16
            activation_fn(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),  # Output: 32x32x3
            nn.Sigmoid()  # Ensure output is in [0, 1]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define Weight Clipping Function
def clip_weights(model, min_val=-0.5, max_val=0.5):
    with torch.no_grad():
        for param in model.parameters():
            param.clamp_(min_val, max_val)

# Load and Preprocess SVHN Dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Training Function with Early Stopping
def train_autoencoder(model, train_loader, test_loader, optimizer, criterion, epochs=100, use_sparsity=False, sparsity_lambda=0.01, patience=10):
    train_losses, test_losses = [], []
    best_psnr = 0.0
    best_test_loss = float('inf')
    epochs_no_improve = 0  # Counter for epochs without improvement

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for batch in train_loader:
            images, _ = batch
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, images)
            
            if use_sparsity:
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss += sparsity_lambda * l1_norm
            
            loss.backward()
            optimizer.step()
            clip_weights(model)
            epoch_loss += loss.item()
        
        model.eval()
        test_loss = 0.0
        psnr_score = 0.0
        with torch.no_grad():
            for batch in test_loader:
                images, _ = batch
                outputs = model(images)
                test_loss += criterion(outputs, images).item()
                for i in range(images.size(0)):
                    psnr_score += psnr(images[i].cpu().numpy(), outputs[i].cpu().numpy(), data_range=1.0)
        
        epoch_loss /= len(train_loader)
        test_loss /= len(test_loader)
        psnr_score /= len(test_dataset)
        
        # Check for improvement in test loss
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            epochs_no_improve = 0  # Reset counter
        else:
            epochs_no_improve += 1  # Increment counter
        
        # Early stopping condition
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1} as test loss did not improve for {patience} epochs.")
            break
        
        if psnr_score > best_psnr:
            best_psnr = psnr_score
        
        train_losses.append(epoch_loss)
        test_losses.append(test_loss)
        
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_loss:.4f}, Test Loss: {test_loss:.4f}, PSNR: {psnr_score:.4f}")
    
    print(f"Best PSNR: {best_psnr:.4f}")
    return train_losses, test_losses, best_psnr

# Train and Compare Models
activation_fns = [nn.ReLU, nn.LeakyReLU]
optimizers = [optim.Adam, optim.RMSprop]
epochs = 10
sparsity_lambda = 0.01
patience = 10  # Early stopping patience

results = {}
for activation_fn in activation_fns:
    for optimizer_fn in optimizers:
        print(f"Training with {activation_fn.__name__} and {optimizer_fn.__name__}")
        
        # Model 1: Weight clipping only
        model1 = ConvAutoencoder(activation_fn)
        optimizer1 = optimizer_fn(model1.parameters(), lr=0.001)
        criterion = nn.MSELoss()
        train_losses1, test_losses1, psnr1 = train_autoencoder(model1, train_loader, test_loader, optimizer1, criterion, epochs, patience=patience)
        
        # Model 2: Weight clipping + L1 sparsity
        model2 = ConvAutoencoder(activation_fn)
        optimizer2 = optimizer_fn(model2.parameters(), lr=0.001)
        train_losses2, test_losses2, psnr2 = train_autoencoder(model2, train_loader, test_loader, optimizer2, criterion, epochs, use_sparsity=True, sparsity_lambda=sparsity_lambda, patience=patience)
        
        key = f"{activation_fn.__name__}_{optimizer_fn.__name__}"
        results[key] = {
            "Weight Clipping Only": {"Train Loss": train_losses1, "Test Loss": test_losses1, "PSNR": psnr1},
            "Weight Clipping + L1 Sparsity": {"Train Loss": train_losses2, "Test Loss": test_losses2, "PSNR": psnr2}
        }

# Compare results
for key, value in results.items():
    print(f"Configuration: {key}")
    print(f"Weight Clipping Only - Best PSNR: {value['Weight Clipping Only']['PSNR']:.4f}")
    print(f"Weight Clipping + L1 Sparsity - Best PSNR: {value['Weight Clipping + L1 Sparsity']['PSNR']:.4f}")
    print()


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
Training with ReLU and Adam
Epoch [1/10], Train Loss: 0.0059, Test Loss: 0.0019, PSNR: 28.9237
Epoch [2/10], Train Loss: 0.0015, Test Loss: 0.0011, PSNR: 31.5255
Epoch [3/10], Train Loss: 0.0011, Test Loss: 0.0009, PSNR: 31.9981
Epoch [4/10], Train Loss: 0.0009, Test Loss: 0.0008, PSNR: 33.0110
Epoch [5/10], Train Loss: 0.0008, Test Loss: 0.0007, PSNR: 33.4674
Epoch [6/10], Train Loss: 0.0006, Test Loss: 0.0006, PSNR: 33.4732
Epoch [7/10], Train Loss: 0.0005, Test Loss: 0.0004, PSNR: 35.5056
Epoch [8/10], Train Loss: 0.0005, Test Loss: 0.0004, PSNR: 35.8557
Epoch [9/10], Train Loss: 0.0004, Test Loss: 0.0007, PSNR: 32.0606
Epoch [10/10], Train Loss: 0.0004, Test Loss: 0.0005, PSNR: 33.5775
Best PSNR: 35.8557
Epoch [1/10], Train Loss: 0.4459, Test Loss: 0.0523, P